In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os

df0 = pd.read_excel(r'C:\Users\Administrador\Desktop\projeto_fsc\base_excel\Base_FlahScore.xlsx')
print(f'jogos na base : {len(df0)}')
# Conversão de tipos de colunas
df0["FT_Goals_H"] = df0["FT_Goals_H"].astype(int)
df0["FT_Goals_A"] = df0["FT_Goals_A"].astype(int)
df0["HT_Goals_H"] = df0["HT_Goals_H"].astype(int)
df0["HT_Goals_A"] = df0["HT_Goals_A"].astype(int)

# Cálculo de Total_Goals_FT e Total_Goals_HT
df0["Total_Goals_FT"] = df0["FT_Goals_H"] + df0["FT_Goals_A"]
df0["Total_Goals_HT"] = df0["HT_Goals_H"] + df0["HT_Goals_A"]

# Cálculo de Total_2HT
df0['Total_2HT'] = df0["Total_Goals_FT"] - df0["Total_Goals_HT"]

# Médias móveis de Total_2HT
df0["Media_Total_2HT_H"] = df0.groupby("Home")["Total_2HT"].rolling(window=10, min_periods=2).mean().reset_index(0, drop=True)
df0["Media_Total_2HT_A"] = df0.groupby("Away")["Total_2HT"].rolling(window=10, min_periods=2).mean().reset_index(0, drop=True)
df0["Media_Total_2HT_H"] = df0.groupby("Home")["Media_Total_2HT_H"].shift(1)
df0["Media_Total_2HT_A"] = df0.groupby("Away")["Media_Total_2HT_A"].shift(1)

# Desvio padrão das médias de Total_2HT
df0['DesvPad_Media_Total_2HT_H'] = df0.groupby('Home')['Media_Total_2HT_H'].rolling(window=10, min_periods=2).std().reset_index(0, drop=True)
df0['DesvPad_Media_Total_2HT_A'] = df0.groupby('Away')['Media_Total_2HT_A'].rolling(window=10, min_periods=2).std().reset_index(0, drop=True)
df0['DesvPad_Media_Total_2HT_H'] = df0.groupby('Home')['DesvPad_Media_Total_2HT_H'].shift(1)
df0['DesvPad_Media_Total_2HT_A'] = df0.groupby('Away')['DesvPad_Media_Total_2HT_A'].shift(1)

# Coeficiente de variação das médias de Total_2HT
df0['CV_Media_Total_2HT_H'] = df0['DesvPad_Media_Total_2HT_H'] / df0['Media_Total_2HT_H']
df0['CV_Media_Total_2HT_A'] = df0['DesvPad_Media_Total_2HT_A'] / df0['Media_Total_2HT_A']

# Resultados e Pontos
df0.loc[df0["FT_Goals_H"] > df0["FT_Goals_A"], "FT_Result"] = "H"
df0.loc[df0["FT_Goals_H"] == df0["FT_Goals_A"], "FT_Result"] = "D"
df0.loc[df0["FT_Goals_H"] < df0["FT_Goals_A"], "FT_Result"] = "A"

df0.loc[df0["FT_Goals_H"] > df0["FT_Goals_A"], "Pontos_H"] = 3
df0.loc[df0["FT_Goals_H"] == df0["FT_Goals_A"], "Pontos_H"] = 1
df0.loc[df0["FT_Goals_H"] < df0["FT_Goals_A"], "Pontos_H"] = 0

df0.loc[df0["FT_Goals_H"] < df0["FT_Goals_A"], "Pontos_A"] = 3
df0.loc[df0["FT_Goals_H"] == df0["FT_Goals_A"], "Pontos_A"] = 1
df0.loc[df0["FT_Goals_H"] > df0["FT_Goals_A"], "Pontos_A"] = 0
# jogos != de 0 x 0
df0['Count_0x0_FT'] = df0.apply(lambda row: 1 if row['Total_Goals_FT'] != 0 else 0, axis=1)
df0['Diferente_0x0_Home'] = (df0.groupby('Home')['Count_0x0_FT'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Diferente_0x0_Away'] = (df0.groupby('Away')['Count_0x0_FT'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)


# Pontos por Jogo (PPJ)
df0["PPJ_H"] = df0.groupby("Home")["Pontos_H"].rolling(window=10, min_periods=2).mean().reset_index(0, drop=True)
df0["PPJ_A"] = df0.groupby("Away")["Pontos_A"].rolling(window=10, min_periods=2).mean().reset_index(0, drop=True)
df0["SG_H"] = df0["FT_Goals_H"] - df0["FT_Goals_A"]
df0["SG_A"] = df0["FT_Goals_A"] - df0["FT_Goals_H"]

df0["PPJ_H"] = df0.groupby("Home")["PPJ_H"].shift(1)
df0["PPJ_A"] = df0.groupby("Away")["PPJ_A"].shift(1)
df0["SG_H"] = df0.groupby("Home")["SG_H"].shift(1)
df0["SG_A"] = df0.groupby("Away")["SG_A"].shift(1)

df0['DP_Pontos_H'] = df0.groupby('Home')['Pontos_H'].rolling(window=10, min_periods=2).std().reset_index(0,drop=True)
df0['DP_Pontos_A'] = df0.groupby('Away')['Pontos_A'].rolling(window=10, min_periods=2).std().reset_index(0,drop=True)

df0['DP_Pontos_H'] = df0.groupby('Home')['DP_Pontos_H'].shift(1)
df0['DP_Pontos_A'] = df0.groupby('Away')['DP_Pontos_A'].shift(1)

df0['CV_Pontos_H'] = (df0['DP_Pontos_H'] / df0['PPJ_H'])
df0['CV_Pontos_A'] = (df0['DP_Pontos_A'] / df0['PPJ_A'])

# Gols Marcados e Sofridos
df0["GM_H"] = df0.FT_Goals_H
df0["GM_A"] = df0.FT_Goals_A
df0["GS_H"] = df0.FT_Goals_A
df0["GS_A"] = df0.FT_Goals_H

# Médias Móveis de Gols Marcados e Sofridos
df0["Media_GM_H"] = df0.groupby("Home")["GM_H"].rolling(window=5, min_periods=2).mean().reset_index(0, drop=True)
df0["Media_GM_A"] = df0.groupby("Away")["GM_A"].rolling(window=5, min_periods=2).mean().reset_index(0, drop=True)
df0["Media_GM_H"] = df0.groupby("Home")["Media_GM_H"].shift(1)
df0["Media_GM_A"] = df0.groupby("Away")["Media_GM_A"].shift(1)

df0["Media_GS_H"] = df0.groupby("Home")["GS_H"].rolling(window=5, min_periods=2).mean().reset_index(0, drop=True)
df0["Media_GS_A"] = df0.groupby("Away")["GS_A"].rolling(window=5, min_periods=2).mean().reset_index(0, drop=True)
df0["Media_GS_H"] = df0.groupby("Home")["Media_GS_H"].shift(1)
df0["Media_GS_A"] = df0.groupby("Away")["Media_GS_A"].shift(1)

# Médias Móveis de Gols Marcados e Sofridos (window=10)
df0["Media_GM_H_10"] = df0.groupby("Home")["GM_H"].rolling(window=10, min_periods=5).mean().reset_index(0, drop=True)
df0["Media_GM_A_10"] = df0.groupby("Away")["GM_A"].rolling(window=10, min_periods=5).mean().reset_index(0, drop=True)
df0["Media_GM_H_10"] = df0.groupby("Home")["Media_GM_H_10"].shift(1)
df0["Media_GM_A_10"] = df0.groupby("Away")["Media_GM_A_10"].shift(1)

df0["Media_GS_H_10"] = df0.groupby("Home")["GS_H"].rolling(window=10, min_periods=5).mean().reset_index(0, drop=True)
df0["Media_GS_A_10"] = df0.groupby("Away")["GS_A"].rolling(window=10, min_periods=5).mean().reset_index(0, drop=True)
df0["Media_GS_H_10"] = df0.groupby("Home")["Media_GS_H_10"].shift(1)
df0["Media_GS_A_10"] = df0.groupby("Away")["Media_GS_A_10"].shift(1)

df0['Over05_HT'] = df0.apply(lambda row: 1 if (row['Total_Goals_HT'] > 0) else 0, axis=1)
df0['Over15_HT'] = df0.apply(lambda row: 1 if (row['Total_Goals_HT'] > 1) else 0, axis=1)
df0['Under05_HT'] = df0.apply(lambda row: 1 if (row['Total_Goals_HT'] < 1) else 0, axis=1)
df0['Over25_FT'] = df0.apply(lambda row: 1 if (row['Total_Goals_FT'] > 2) else 0, axis=1) 
df0['Home_Win'] = df0.apply(lambda row: 1 if row['FT_Goals_H'] > row['FT_Goals_A'] else 0, axis=1)
df0['Away_Win'] = df0.apply(lambda row: 1 if row['FT_Goals_A'] > row['FT_Goals_H'] else 0, axis=1)
df0['Home_Win_HT'] = df0.apply(lambda row: 1 if row['HT_Goals_H'] > row['HT_Goals_A'] else 0, axis=1)
df0['Away_Win_HT'] = df0.apply(lambda row: 1 if row['HT_Goals_A'] > row['HT_Goals_H'] else 0, axis=1)
df0['BTTS'] = df0.apply(lambda row: 1 if (row['FT_Goals_H'] > 0 and row['FT_Goals_A'] > 0) else 0, axis=1) 

df0['Porc_Over05HT_Home'] = (df0.groupby('Home')['Over05_HT'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Porc_Over05HT_Away'] = (df0.groupby('Away')['Over05_HT'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Porc_Over15HT_Home'] = (df0.groupby('Home')['Over15_HT'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Porc_Over15HT_Away'] = (df0.groupby('Away')['Over15_HT'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Porc_Under05HT_Home'] = (df0.groupby('Home')['Under05_HT'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Porc_Under05HT_Away'] = (df0.groupby('Away')['Under05_HT'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Porc_Home_Win'] = (df0.groupby('Home')['Home_Win'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Porc_Away_Win'] = (df0.groupby('Away')['Away_Win'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Porc_Home_Win_HT'] = (df0.groupby('Home')['Home_Win_HT'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Porc_Away_Win_HT'] = (df0.groupby('Away')['Away_Win_HT'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Porc_Over25FT_Home'] = (df0.groupby('Home')['Over25_FT'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Porc_Over25FT_Away'] = (df0.groupby('Away')['Over25_FT'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Porc_BTTS_Home'] = (df0.groupby('Home')['BTTS'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)
df0['Porc_BTTS_Away'] = (df0.groupby('Away')['BTTS'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True) * 100).shift(1)


df_H = df0[
    [
        "Home",
        "Away",
        "Media_GM_H",
        "Media_GS_H",
        "Media_GM_H_10",
        "Media_GS_H_10",
        "Media_GM_A",
        "Media_GS_A",
        "Media_GM_A_10",
        "Media_GS_A_10",
        "Media_Total_2HT_H",
        "CV_Media_Total_2HT_H",
        "PPJ_H",
        "CV_Pontos_H",
        "Porc_Over05HT_Home",
        "Porc_Over15HT_Home",
        "Porc_Under05HT_Home",
        "Porc_Home_Win",
        "Porc_Home_Win_HT",
        "Porc_Over25FT_Home",
        "Porc_BTTS_Home",
        "Diferente_0x0_Home",
    ]
]
df_A = df0[
    [
        "Home",
        "Away",
        "Media_GM_A",
        "Media_GS_A",
        "Media_GM_A_10",
        "Media_GS_A_10",
        "Media_GM_H",
        "Media_GS_H",
        "Media_GM_H_10",
        "Media_GS_H_10",
        "Media_Total_2HT_A",
        "CV_Media_Total_2HT_A",
        "PPJ_A",
        "CV_Pontos_A",
        "Porc_Over05HT_Away",
        "Porc_Over15HT_Away",
        "Porc_Under05HT_Away",
        "Porc_Away_Win",
        "Porc_Away_Win_HT",
        "Porc_Over25FT_Away",
        "Porc_BTTS_Away",
        "Diferente_0x0_Away",
    ]
]

jogos na base : 6738


In [2]:
jogos_do_dia = pd.read_excel(r"C:\Users\Administrador\Desktop\projeto_fsc\base_excel\jogos_do_dia_trat.xlsx")
filtered_df = jogos_do_dia.copy()
filtered_df["Date"] = pd.to_datetime(filtered_df["Date"], format="%d/%m/%Y")
filtered_df["Date"] = filtered_df["Date"].dt.date
#date_to_filter = "2023-11-03"
#filtered_df = filtered_df[filtered_df["Date"] == date_to_filter]
jogo_lista = []

for index, row in filtered_df.iterrows():
    id = row["Id"]
    League = row["League"]
    Date = row["Date"]
    Time = row['Time']
    Round =row['Round']
    home = row["Home"]
    away = row["Away"]
    colocacao_home = row['posicao_home']
    colocacao_away = row['posicao_away']    
    FT_Odd_H = row["FT_Odd_H"]
    FT_Odd_D = row["FT_Odd_D"]
    FT_Odd_A = row["FT_Odd_A"]
    FT_Odd_Over25 = row["FT_Odd_Over25"]
    FT_Odd_Under25 = row["FT_Odd_Under25"]
    FT_Odd_BTTS_Yes = row["Odd_BTTS_Yes"]
    FT_Odd_BTTS_No = row["Odd_BTTS_No"]

    try:
        # Filtra os DataFrames df_H e df_A
        df1 = df_H[df_H.Home == home].tail(1)
        df2 = df_A[df_A.Away == away].tail(1)

        # Calcula as médias de gols diretamente
        Gols_Marcados_Home = round(df1["Media_GM_H"].iloc[0], 2)
        Gols_Sofridos_Home = round(df1["Media_GS_H"].iloc[0], 2)
        Gols_Marcados_Home_10 = round(df1["Media_GM_H_10"].iloc[0], 2)
        Gols_Sofridos_Home_10 = round(df1["Media_GS_H_10"].iloc[0], 2)
        Media_Total_2HT_H = round(df1["Media_Total_2HT_H"].iloc[0], 2)
        CV_Media_Total_2HT_H = round(df1["CV_Media_Total_2HT_H"].iloc[0], 2)
        PPJ_H = round(df1["PPJ_H"].iloc[0], 2)
        CV_Pontos_H = round(df1["CV_Pontos_H"].iloc[0], 2)
        Porc_Over05HT_Home= round(df1["Porc_Over05HT_Home"].iloc[0], 2)
        Porc_Over15HT_Home= round(df1["Porc_Over15HT_Home"].iloc[0], 2)
        Porc_Under05HT_Home= round(df1["Porc_Under05HT_Home"].iloc[0], 2)
        Porc_Home_Win = round(df1['Porc_Home_Win'].iloc[0], 2)
        Porc_Home_Win_HT = round(df1['Porc_Home_Win_HT'].iloc[0], 2)
        Porc_Over25FT_Home = round(df1['Porc_Over25FT_Home'].iloc[0], 2)
        Porc_BTTS_Home = round(df1['Porc_BTTS_Home'].iloc[0], 2)     
        Diferente_0x0_Home = round(df1['Diferente_0x0_Home'].iloc[0], 2)   
                  
        #........................................................................................                                                       
        Gols_Marcados_Away = round(df2["Media_GM_A"].iloc[0], 2)
        Gols_Sofridos_Away = round(df2["Media_GS_A"].iloc[0], 2)
        Gols_Marcados_Away_10 = round(df2["Media_GM_A_10"].iloc[0], 2)
        Gols_Sofridos_Away_10 = round(df2["Media_GS_A_10"].iloc[0], 2)
        Media_Total_2HT_A = round(df2["Media_Total_2HT_A"].iloc[0], 2)
        CV_Media_Total_2HT_A = round(df2["CV_Media_Total_2HT_A"].iloc[0], 2)
        PPJ_A = round(df2["PPJ_A"].iloc[0], 2)
        CV_Pontos_A = round(df2["CV_Pontos_A"].iloc[0], 2)
        Porc_Over05HT_Away= round(df2["Porc_Over05HT_Away"].iloc[0], 2)
        Porc_Over15HT_Away= round(df2["Porc_Over15HT_Away"].iloc[0], 2)
        Porc_Under05HT_Away= round(df2["Porc_Under05HT_Away"].iloc[0], 2)
        Porc_Away_Win = round(df2['Porc_Away_Win'].iloc[0], 2)
        Porc_Away_Win_HT = round(df2['Porc_Away_Win_HT'].iloc[0], 2)
        Porc_Over25FT_Away = round(df2['Porc_Over25FT_Away'].iloc[0], 2)
        Porc_BTTS_Away = round(df2['Porc_BTTS_Away'].iloc[0], 2)    
        Diferente_0x0_Away = round(df2['Diferente_0x0_Away'].iloc[0], 2)   
 
                
        jogo = {
            "Id" : id,
            "League": League,
            "Date": Date,
            "Time": Time,
            'Round': Round,
            "Home": home,
            "Away": away,
            "posicao_home":colocacao_home,
            "posicao_away":colocacao_away,            
            "FT_Odd_H": FT_Odd_H,
            "FT_Odd_D": FT_Odd_D,
            "FT_Odd_A": FT_Odd_A,
            "FT_Odd_Over25": FT_Odd_Over25,
            "FT_Odd_Under25": FT_Odd_Under25,
            "FT_Odd_BTTS_Yes": FT_Odd_BTTS_Yes,
            "FT_Odd_BTTS_No": FT_Odd_BTTS_No,
            "Media_GM_H": Gols_Marcados_Home,
            "Media_GS_H": Gols_Sofridos_Home,
            "Media_GM_H_10": Gols_Marcados_Home_10,
            "Media_GS_H_10": Gols_Sofridos_Home_10,
            "Media_Total_2HT_H": Media_Total_2HT_H,
            'CV_Media_Total_2HT_H': CV_Media_Total_2HT_H,
            'PPJ_H': PPJ_H,
            'CV_Pontos_H': CV_Pontos_H,
            'Porc_Over05HT_Home' :Porc_Over05HT_Home,
            'Porc_Over15HT_Home' :Porc_Over15HT_Home,
            'Porc_Under05HT_Home' :Porc_Under05HT_Home,                        
            'Porc_Home_Win' : Porc_Home_Win,
            'Porc_Home_Win_HT' : Porc_Home_Win_HT,            
            'Porc_Over25FT_Home':Porc_Over25FT_Home,
            'Porc_BTTS_Home' : Porc_BTTS_Home ,   
            'Diferente_0x0_Home': Diferente_0x0_Home,           
            "Media_GM_A": Gols_Marcados_Away,
            "Media_GS_A": Gols_Sofridos_Away,
            "Media_GM_A_10": Gols_Marcados_Away_10,
            "Media_GS_A_10": Gols_Sofridos_Away_10,
            "Media_Total_2HT_A": Media_Total_2HT_A,
            'CV_Media_Total_2HT_A': CV_Media_Total_2HT_A,
            'PPJ_A': PPJ_A,
            'CV_Pontos_A': CV_Pontos_A,
            'Porc_Over05HT_Away':Porc_Over05HT_Away,
            'Porc_Over15HT_Away' :Porc_Over15HT_Away,
            'Porc_Under05HT_Away' :Porc_Under05HT_Away,  
            'Porc_Away_Win' : Porc_Away_Win,
            'Porc_Away_Win_HT' : Porc_Away_Win_HT,            
            'Porc_Over25FT_Away':Porc_Over25FT_Away,
            'Porc_BTTS_Away' : Porc_BTTS_Away ,      
             'Diferente_0x0_Away': Diferente_0x0_Away,                          
        }

        jogo_lista.append(jogo)
    except:
        pass

df = pd.DataFrame(jogo_lista)
df.index = df.index + 1
df.index.name = "Nº"

##########################################################################################################################################
df["xg_home"] = (np.sqrt(df["Media_GM_H"] * df["Media_GS_A"])).round(2)
df["xg_away"] = (np.sqrt(df["Media_GM_A"] * df["Media_GS_H"])).round(2)
df["xg_jogo"] = (df["xg_home"] + df["xg_away"]).round(2)


print(len(df))



diretorio_w = r'C:\Users\Administrador\Desktop\repositorios\goalpyapp\dados_csv'
if not os.path.exists(diretorio_w):
    os.makedirs(diretorio_w)

df.to_csv(os.path.join(diretorio_w, "proximos_jogos.csv"), index=False)

df

229


,Id,League,Date,Time,Round,Home,Away,posicao_home,posicao_away,FT_Odd_H,...,Porc_Over15HT_Away,Porc_Under05HT_Away,Porc_Away_Win,Porc_Away_Win_HT,Porc_Over25FT_Away,Porc_BTTS_Away,Diferente_0x0_Away,xg_home,xg_away,xg_jogo
Nº,,,,,,,,,,,,,,,,,,,,,
1,rHiFR4Ta,SUÉCIA - ALLSVENSKAN,2024-04-06,14:00,RODADA 2,Brommapojkarna,AIK Stockholm,4.0,11.0,NaN,...,60.00,20.00,10.00,10.00,60.00,70.00,100.00,1.67,0.75,2.42
2,pMPlBqqJ,SUÉCIA - ALLSVENSKAN,2024-04-06,14:00,RODADA 2,Djurgarden,Hacken,9.0,1.0,NaN,...,10.00,30.00,20.00,20.00,60.00,60.00,100.00,1.50,0.49,1.99
3,jNShWwQq,ÁUSTRIA - BUNDESLIGA,2024-02-10,13:00,RODADA 18,Altach,BW Linz,10.0,9.0,2.24,...,57.14,28.57,28.57,28.57,57.14,42.86,100.00,1.62,0.40,2.02
4,lEtRzKHS,ÁUSTRIA - BUNDESLIGA,2024-02-10,13:00,RODADA 18,Austria Vienna,Hartberg,8.0,4.0,1.97,...,42.86,42.86,28.57,28.57,57.14,57.14,85.71,1.26,0.00,1.26
5,84YIxtmG,ÁUSTRIA - BUNDESLIGA,2024-02-10,13:00,RODADA 18,LASK,A. Klagenfurt,3.0,5.0,1.60,...,28.57,14.29,57.14,42.86,71.43,42.86,85.71,0.85,0.00,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,zXPPGmFs,SUÉCIA - ALLSVENSKAN,2024-03-30,15:00,RODADA 1,Göteborg,Djurgarden,7.0,9.0,2.80,...,20.00,50.00,20.00,20.00,50.00,50.00,80.00,1.40,0.69,2.09
226,lfWuD5E6,SUÉCIA - ALLSVENSKAN,2024-03-30,15:00,RODADA 1,Hacken,Mjallby,1.0,3.0,1.37,...,30.00,40.00,40.00,20.00,30.00,30.00,90.00,2.18,0.92,3.10
227,b72dYFpF,SUÉCIA - ALLSVENSKAN,2024-03-30,15:00,RODADA 1,Hammarby,Kalmar,12.0,8.0,1.65,...,70.00,20.00,50.00,40.00,70.00,50.00,90.00,1.26,1.40,2.66


In [6]:
dfr= pd.read_csv(r'C:\Users\Administrador\Desktop\repositorios\goalpyapp\dados_csv\proximos_jogos_RR.csv')

print(list(dfr.columns))

['League', 'Date', 'Time', 'Round', 'Home', 'Away', 'Class_Home', 'Class_Away', 'FT_Odd_H', 'FT_Odd_D', 'FT_Odd_A', 'HT_Odd_Over05', 'HT_Odd_Under05', 'FT_Odd_Over15', 'FT_Odd_Under15', 'FT_Odd_Over25', 'FT_Odd_Under25', 'FT_Odd_Over35', 'FT_Odd_Under35', 'FT_Odd_BTTS_Yes', 'FT_Odd_BTTS_No', 'p_H', 'p_D', 'p_A', 'CV_HDA', 'P_expt_H', 'P_expt_A', 'Media_GM_H_6j', 'Media_GS_H_6j', 'Media_Total_1HT_H', 'CV_Media_Total_1HT_H', 'Media_Total_2HT_H', 'CV_Media_Total_2HT_H', 'Media_primeiro_gol_home', 'Media_primeiro_gol_home_sofrido', 'Porc_Marcou_Primeiro_Golo_H', 'Porc_Sofreu_Primeiro_Golo_H', 'Porc_Marcou_Primeiro_Golo_H_1P', 'Porc_Sofreu_Primeiro_Golo_H_1P', 'media_movel_chutes_por_gol_H', 'media_chutes_home', 'PPJ_H', 'CV_Pontos_H', 'Power_home', 'Media_CG_H', 'CV_CG_H', 'Media_VG_H', 'CV_VG_H', 'Porc_Over05HT_Home', 'Porc_Over15HT_Home', 'Porc_Under05HT_Home', 'Porc_Home_Win', 'Porc_Over15FT_Home', 'Porc_Over25FT_Home', 'Porc_BTTS_Home', 'Media_FT_Corners_H_MAR', 'Media_FT_Corners_H_SFR